In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from vincenty import vincenty

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
#Import natural disaster data
raw = pd.read_csv('natural_disaster_human_mobility.csv')

In [33]:
#Switch latitude and longtitude columns, rename and reorder, dataframe
df = raw
df = df.rename(columns = {'disaster.event': 'event', 'user.anon': 'user', 'latitude': 'long', 'longitude.anon': 'lat'})
df = df[['event', 'user', 'lat', 'long', 'time']]
df['time'] = pd.DatetimeIndex(df['time'])


In [34]:
#Calculate event location by averaging the first lat long entry for each user
group = df.groupby(by = 'event').mean()
group

,user,lat,long
event,,,
01_Wipha,24368.338734,28.391620,139.668645
02_Halong,2482.137684,43.006657,127.762166
06_Kalmaegi,591.625357,5.342795,120.410101
08_Rammasun_Manila,17102.932522,9.572405,120.997554
12_Bohol,3805.313771,-2.370050,123.860556
13_Iquique,781.068314,-18.004903,-70.133271
14_Napa,888.637865,34.441945,-122.302310
21_Norfolk,4276.517602,69.428469,1.132364
22_Hamburg,1428.430413,63.611212,10.002712


In [37]:
#Export average coordinates to csv for tableau
group.to_csv('ave_coord.csv')

In [243]:
napa2 = df[(df['event'] == '14_Napa')]
napa2.shape
napa2.to_csv('napa2.csv')

In [183]:
napa = df[(df['event'] == '14_Napa') & (df['user'] == 3)]

In [189]:
#Generate distance column
napa['lat_1'] = napa['lat'].shift()
napa['long_1'] = napa['long'].shift()
napa['dist_mi'] = napa.apply(lambda row: vincenty((row['lat'], row['long']), (row['lat_1'], row['long_1'])), axis=1)
napa['dist_ft'] = napa['dist_mi']*5260

/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [229]:
napa['time_diff'] = napa['time'].diff()
napa['time_diff_hour'] = napa['time_diff']/np.timedelta64(1, 'h')
napa['time_diff_day'] = napa['time_diff']/np.timedelta64(1, 'D')

/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [231]:
napa['velocity_mph'] = napa['dist_mi']/napa['time_diff_hour'] 

/Users/mattcarney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [238]:
m = napa['time'].min()
m

Timestamp('2014-07-30 13:05:52')

In [233]:
napa.to_csv('napa.csv')

In [239]:
napa.iloc[0,:]

event                         14_Napa
user                                3
lat                           34.4481
long                         -122.311
time              2014-07-30 13:05:52
lat_1                             NaN
long_1                            NaN
dist                              NaN
dist_mi                           NaN
dist_ft                           NaN
time_diff                         NaT
time_diff_hour                    NaN
time_diff_day                     NaN
velocity_mph                      NaN
Name: 2579283, dtype: object